## Assignment 2: Gender Income Gap

This notebook presents a an analysis of the gender income gap based on the US Census data from

http://www.census.gov/hhes/www/cpstables/032015/perinc/pinc03_000.htm

We will be be using the data to try to build a -morally questionable- model to support the decission on "what to offer to a woman job candidate, given what is offered to a man for the same job?"

This notebook is based on the skeleton notebook from FBB:

https://github.com/fedhere/PUI2016_fb55/blob/master/HW5_fb55/genderIncomeBias.ipynb

In [5]:
# Standard imports to work with datasets, plots, distributions, etc.
import pylab as pl
import pandas as pd
import numpy as np
import os

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [30]:
# Defining structures and getting the data
# We will create two dictionaries to store all data (allmen and allwomen)

# Base URL for the data store
url = "http://cosmo.nyu.edu/~fb55/PUI2016/data/"

# File Coding
# 08_2 are male,
# 08_3 are female,
# _1_1 is all ethnicities
# _1_4 is white
# _1_6 is black
# _1_8 is asian
# _1_9 is hispanic

# Using the pandas read from excel method to create the dataframes
# Will read directly the xls files, skipping the first eight rows

dfMenAll = pd.read_excel(url + 'pinc08_2_1_1.xls', skiprows=[0,1,2,3,4,5,6,7,8])
dfMenWhite = pd.read_excel(url + 'pinc08_2_1_4.xls', skiprows=[0,1,2,3,4,5,6,7,8])
dfMenBlack = pd.read_excel(url + 'pinc08_2_1_6.xls', skiprows=[0,1,2,3,4,5,6,7,8])
dfMenHispanic = pd.read_excel(url + 'pinc08_2_1_8.xls', skiprows=[0,1,2,3,4,5,6,7,8])
dfMenAsian = pd.read_excel(url + 'pinc08_2_1_9.xls', skiprows=[0,1,2,3,4,5,6,7,8])

dfWomenAll = pd.read_excel(url + 'pinc08_3_1_1.xls', skiprows=[0,1,2,3,4,5,6,7,8])
dfWomenWhite = pd.read_excel(url + 'pinc08_3_1_4.xls', skiprows=[0,1,2,3,4,5,6,7,8])
dfWomenBlack = pd.read_excel(url + 'pinc08_3_1_6.xls', skiprows=[0,1,2,3,4,5,6,7,8])
dfWomenHispanic = pd.read_excel(url + 'pinc08_3_1_8.xls', skiprows=[0,1,2,3,4,5,6,7,8])
dfWomenAsian= pd.read_excel(url + 'pinc08_3_1_9.xls', skiprows=[0,1,2,3,4,5,6,7,8])

allmales = {'all':dfMenAll, 'white':dfMenWhite, 'black':dfMenBlack, 'hispanic':dfMenHispanic, 'asian':dfMenAsian}
allfemales = {'all':dfWomenAll, 'white':dfWomenWhite, 'black':dfWomenBlack, 
             'hispanic':dfWomenHispanic, 'asian':dfWomenAsian}

allfemales['all'].head()

,Characteristic,Total with Income,"$1 to $2,499 or loss","$2,500 to $4,999","$5,000 to $7,499","$7,500 to $9,999","$10,000 to $12,499","$12,500 to $14,999","$15,000 to $17,499","$17,500 to $19,999",...,"$92,500 to $94,999","$95,000 to $97,499","$97,500 to $99,999","$100,000 and over",Median income,Unnamed: 44,Mean income,Unnamed: 46,Gini ratio,Unnamed: 48
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Value\n(Dol.),Standard\nerror (Dol.),Value\n(Dol.),Standard\nerror (Dol.),Ratio,Standard\nerror
1,....Total,109956.0,7590.0,4298.0,5557.0,7874.0,8052.0,5720.0,6768.0,4377.0,...,167.0,285.0,184.0,4442.0,22063,137,31968,250,0.495,0.0025
2,Earnings,74545.0,4198.0,3118.0,3269.0,2662.0,4579.0,1994.0,4134.0,2585.0,...,60.0,236.0,127.0,3535.0,27736,364,36189,307,0.466,0.0032
3,..Wages and Salary,71220.0,3835.0,2954.0,3005.0,2411.0,4309.0,1941.0,3908.0,2484.0,...,61.0,233.0,126.0,3370.0,28402,379,36412,316,0.461,0.0032
4,..Nonfarm Self-Employment,4694.0,1282.0,288.0,410.0,253.0,388.0,84.0,265.0,126.0,...,0.0,0.0,0.0,127.0,10732,399,20942,1123,0.65,0.0162


In [29]:
# the files are a bit of a mess! 
allfemales['all'].columns

# just run this cell to get an idea of the data
for k,v in allfemales.items():
    print("#####", k)
    print(v.describe())
for k,v in allmales.items():
    print("#####", k)
    print(v.describe())


('#####', 'hispanic')
       Total with Income  $1 to $2,499 or loss  $2,500 to $4,999  \
count          46.000000             46.000000         46.000000   
mean          809.478261            150.347826         42.413043   
std          1476.967354            439.113765         59.593652   
min             2.000000              0.000000          0.000000   
25%                  NaN                   NaN               NaN   
50%                  NaN                   NaN               NaN   
75%                  NaN                   NaN               NaN   
max          5648.000000           2178.000000        224.000000   

       $5,000 to $7,499  $7,500 to $9,999  $10,000 to $12,499  \
count         46.000000         46.000000           46.000000   
mean          53.934783         59.434783           56.717391   
std           82.427989         95.528623           93.491215   
min            0.000000          0.000000            0.000000   
25%                 NaN               Na

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


       Total with Income  $1 to $2,499 or loss  $2,500 to $4,999  \
count          46.000000             46.000000         46.000000   
mean         2120.260870            292.304348        141.021739   
std          3673.947011            563.958859        181.001595   
min             0.000000              0.000000          0.000000   
25%                  NaN                   NaN               NaN   
50%                  NaN                   NaN               NaN   
75%                  NaN                   NaN               NaN   
max         14346.000000           2727.000000        679.000000   

       $5,000 to $7,499  $7,500 to $9,999  $10,000 to $12,499  \
count         46.000000         46.000000           46.000000   
mean         140.934783        229.456522          179.021739   
std          203.026917        371.274843          321.057730   
min            0.000000          0.000000            0.000000   
25%                 NaN               NaN                 NaN 